In [1]:
from math import sin, cos, radians
import cv2
import os
import re
import yaml
import numpy as np

In [2]:
def get_config(key) -> int:
    with open('./conf/config.yaml') as file:
        file = yaml.load(file, Loader=yaml.FullLoader)

    return file[key]

config = get_config('CONFIG')

In [3]:
directory = 'data/raw/'

path_train = 'data/faces/train/'
path_test = 'data/faces/test/'

reg_person = 'person\\d\\d'
reg_filename = 'person.*'

def clear_directories():
    for filename in os.scandir(directory):
        if filename.is_file() and '.DS_Store' not in filename.path:
            try:
                imagePath = filename.path
                new_filename = path_train+'{}/{}'.format(re.search(reg_person, imagePath)[0],re.search(reg_filename, imagePath)[0])
                if 'classes' in imagePath:
                    new_filename = path_test+'classes/{}'.format(re.search(reg_filename, imagePath)[0])
                if 'samples' in imagePath:
                    new_filename = path_test+'samples/{}'.format(re.search(reg_filename, imagePath)[0])
                
                print('file removed: ', new_filename)
                os.remove(new_filename)
                
            except FileNotFoundError:
                print('no such a file: ',new_filename)
                
            try:
                flipped_filename = new_filename.replace('.jpg','_h.jpg')
                print('file removed: ', flipped_filename)
                os.remove(flipped_filename)
            except FileNotFoundError:
                print('no such a file: ',flipped_filename)

In [4]:
clear_directories()

file removed:  data/faces/test/classes/person11_classes.jpg
file removed:  data/faces/test/classes/person11_classes_h.jpg
no such a file:  data/faces/test/classes/person11_classes_h.jpg
file removed:  data/faces/test/samples/person03_samples.jpg
file removed:  data/faces/test/samples/person03_samples_h.jpg
no such a file:  data/faces/test/samples/person03_samples_h.jpg
file removed:  data/faces/test/samples/person16_samples.jpg
file removed:  data/faces/test/samples/person16_samples_h.jpg
no such a file:  data/faces/test/samples/person16_samples_h.jpg
file removed:  data/faces/test/samples/person13_samples.jpg
file removed:  data/faces/test/samples/person13_samples_h.jpg
no such a file:  data/faces/test/samples/person13_samples_h.jpg
file removed:  data/faces/test/classes/person01_classes.jpg
file removed:  data/faces/test/classes/person01_classes_h.jpg
no such a file:  data/faces/test/classes/person01_classes_h.jpg
file removed:  data/faces/test/classes/person14_classes.jpg
file remov

In [5]:
def scaled(scale, img):
    #calculate the 50 percent of original dimensions
    width = int(img.shape[1] * scale / 100)
    height = int(img.shape[0] * scale / 100)
    return cv2.resize(img, [width,height])

def rotate_image(image, angle):
    if angle == 0: return image
    height, width = image.shape[:2]
    rot_mat = cv2.getRotationMatrix2D((width/2, height/2), angle, 0.9)
    result = cv2.warpAffine(image, rot_mat, (width, height), flags=cv2.INTER_LINEAR)
    return result

def rotate_point(pos, img, angle):
    if angle == 0: return pos
    x = pos[0] - img.shape[1]*0.4
    y = pos[1] - img.shape[0]*0.4
    newx = x*cos(radians(angle)) + y*sin(radians(angle)) + img.shape[1]*0.4
    newy = -x*sin(radians(angle)) + y*cos(radians(angle)) + img.shape[0]*0.4
    return int(newx), int(newy), pos[2], pos[3]

In [6]:
cascade_algos = ["conf/haarcascade_frontalface_alt_two.xml",'conf/haarcascade_profileface.xml']
settings = {
    'scaleFactor': 1.1, 
    'minNeighbors': 3, 
    'minSize': (128, 128), 
    'flags': cv2.CASCADE_FIND_BIGGEST_OBJECT|cv2.CASCADE_DO_ROUGH_SEARCH,
    'outputRejectLevels': True
}

for filename in os.scandir(directory):
    found = False
    if filename.is_file() and '.DS_Store' not in filename.path:
        imagePath = filename.path
 
        # Read the input image
        img = cv2.imread(imagePath)
        
        img_scaled = scaled(img=img, scale=100)
        
        # Convert into grayscale
        gray_img = cv2.cvtColor(img_scaled, cv2.COLOR_BGR2GRAY)
        
        for cascade in cascade_algos:
            face_cascade = cv2.CascadeClassifier(cascade)
            
            for angle in [0, -10, 10, -25, 25]:
                r_img = rotate_image(gray_img, angle)
                # print(angle)
                faces,_,weights = face_cascade.detectMultiScale3(r_img, **settings)
                if len(faces):
                    faces = [rotate_point(faces[-1], gray_img, -angle)]
                    break
            
            if len(weights)>0 and weights[0] > 55:
                for (x, y, w, h) in faces:
                    # cv2.rectangle(gray_img, (x, y), (x+w, y+h), (255, 0, 0), 2)
                    gray_img = gray_img[y:y + h, x:x + w]
                    output_image = cv2.resize(gray_img, config['IMG_SHAPE'][:-1])
                    normalizedImg = np.zeros((800, 800))
                    output_image = cv2.normalize(output_image,  normalizedImg, 0, 255, cv2.NORM_MINMAX)
                    # output_image = cv2.equalizeHist(output_image)
                    # output_image = output_image/255.0
                    # output_image = cv2.pow(output_image, 0.7)
                    # output_image = np.uint8(output_image*255)
                    if 'classes' not in imagePath and 'samples' not in imagePath:
                        new_filename = path_train+'{}/{}'.format(re.search(reg_person, imagePath)[0], re.search(reg_filename, imagePath)[0])
                        cv2.imwrite(new_filename.replace('.jpg','_h.jpg'), cv2.flip(output_image, 1))
                        
                    if 'classes' in imagePath:
                        new_filename = path_test+'classes/{}'.format(re.search(reg_filename, imagePath)[0])
                        
                    if 'samples' in imagePath:
                        new_filename = path_test+'samples/{}'.format(re.search(reg_filename, imagePath)[0])

                    cv2.imwrite(new_filename, output_image)

                    print('raw person path: ', imagePath)
                    print('extracted face path: ', new_filename)
                    found = True
                    break
                
                if found:
                    found = False
                    break

raw person path:  data/raw/person11_classes.jpg
extracted face path:  data/faces/test/classes/person11_classes.jpg
raw person path:  data/raw/person03_samples.jpg
extracted face path:  data/faces/test/samples/person03_samples.jpg
raw person path:  data/raw/person16_samples.jpg
extracted face path:  data/faces/test/samples/person16_samples.jpg
raw person path:  data/raw/person13_samples.jpg
extracted face path:  data/faces/test/samples/person13_samples.jpg
raw person path:  data/raw/person01_classes.jpg
extracted face path:  data/faces/test/classes/person01_classes.jpg
raw person path:  data/raw/person14_classes.jpg
extracted face path:  data/faces/test/classes/person14_classes.jpg
raw person path:  data/raw/person05_01.jpg
extracted face path:  data/faces/train/person05/person05_01.jpg
raw person path:  data/raw/person05_samples.jpg
extracted face path:  data/faces/test/samples/person05_samples.jpg
raw person path:  data/raw/person17_classes.jpg
extracted face path:  data/faces/test/cl